<a href="https://colab.research.google.com/github/SenanS/ML-Sign-Recognition/blob/main/Colab_ML_Indiv_Project_checkpoint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Set up Kaggle API & Download dataset (Disabled to stop multiple downloads of same dataset)
import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/-ML"

# %cd /content/drive/MyDrive/-ML/GTSRB-Data/
# !kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign #--unzip
# !unzip /content/drive/MyDrive/-ML/GTSRB-Data/gtsrb-german-traffic-sign.zip -d /content/drive/MyDrive/-ML/GTSRB-Data/

# List the data location
%ls /content/drive/MyDrive/-ML/GTSRB-Data/

archive.zip                    meta/  Meta.csv  Test/     train/  Train.csv
gtsrb-german-traffic-sign.zip  Meta/  test/     Test.csv  Train/


In [3]:
import os
import pandas as pd
import numpy as np
import sklearn
import cv2
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models

from keras.models import load_model
from PIL import Image
from matplotlib import pyplot as plt


In [4]:
# Image variables
im_width  = 244
im_height = 244

# Path Variables
collab_path = '/content/drive/MyDrive/-ML/'
data_path   = collab_path + 'GTSRB-Data/'

In [5]:

print(os.listdir(data_path))

df_meta  = pd.read_csv(data_path + 'Meta.csv', delimiter=',')
df_test  = pd.read_csv(data_path + 'Test.csv', delimiter=',')
df_train = pd.read_csv(data_path + 'Train.csv', delimiter=',')

['Meta.csv', 'Test.csv', 'Train.csv', 'archive.zip', 'gtsrb-german-traffic-sign.zip', 'Meta', 'Test', 'Train', 'meta', 'test', 'train']


In [6]:
print('Number of Classes: ',df_meta.shape[0])

Number of Classes:  43


# TODO:
* Pre-process images to VGG input size and feature range
* Set up Training and validation sets (split training) (Keras Data generators)
    
    

In [7]:
# Lower quality of images by 256 factor & set validation set to 20%
ImDG_train = ImageDataGenerator(rescale=1./255, validation_split=0.2) 
data_train = ImDG_train.flow_from_directory(data_path + 'Train/',
                                            target_size  = (im_width, im_height),
                                            batch_size   = 32,
                                            class_mode   ='categorical',
                                            shuffle      = True,
                                            color_mode   ='rgb',
                                            # interpolation='hamming',
                                            subset       ='training'
                                           )

# # ImDG_valid = ImageDataGenerator(rescale=1./255, validation_split=0.2) 
# ImDG_valid = ImageDataGenerator(validation_split=0.2) 
data_valid = ImDG_train.flow_from_directory(data_path + 'Train/',
                                            target_size  = (im_width, im_height),
                                            batch_size   = 32,
                                            class_mode   ='categorical',
                                            shuffle      = True,
                                            color_mode   ='rgb',
                                            # interpolation='hamming',
                                            subset       ='validation'
                                           )


Found 31368 images belonging to 43 classes.
Found 7841 images belonging to 43 classes.


In [ ]:
test_labels = df_test["ClassId"].values
test_imgs   = df_test["Path"].values

data = []
for img in test_imgs:
    try:
        test_im         = cv2.imread(data_path + img)
        image_fromarray = Image.fromarray(test_im, mode='RGB')
        resize_image    = image_fromarray.resize((im_height, im_width))
        data.append(np.array(resize_image))
    except:
        print("Error in " + img)

data_test = np.array(data)
data_test = data_test/255

In [8]:
# Import model without a classification layer (for transfer learning)
vgg_model = VGG16(weights="imagenet", include_top=False, input_shape=(im_width, im_height, 3))
# Freeze existing model
vgg_model.trainable = False


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
vgg_model.summary()


In [9]:
flatten_layer    = layers.Flatten()
dense_layer1     = layers.Dense(512, activation='relu', kernel_regularizer='l1')
# dense_layer2 = layers.Dense(256, activation='relu')
# dense_layer3 = layers.Dense(256, activation='relu')
prediction_layer = layers.Dense(43, activation='softmax')


model = models.Sequential([
    vgg_model,
    flatten_layer,
    dense_layer1,
    # dense_layer2,
    prediction_layer
])

In [10]:
# from sklearn.utils import class_weight
# class_bal = class_weight.compute_class_weight('balanced', np.unique(df_train["ClassId"].values), df_train["ClassId"].values)
label_count = [np.sum(df_train["ClassId"].values[:] == i) for i in np.unique(df_train["ClassId"].values)]
label_order = np.unique(df_train["ClassId"].values)
data_keys   = data_train.class_indices.keys()
dict = {}
for curr_order, curr_count in zip(label_order, label_count):
    dict[int(curr_order)] = curr_count
# dict(enumerate(label_count))



In [11]:
reclassed_count = [None] * df_meta.shape[0]

for i,j in zip(data_keys, label_order):
    reclassed_count[j] = label_count[int(i)]


reclassed_dict = {}
for label_order, reclassed_count in zip(label_order, reclassed_count):
    reclassed_dict[label_order] = reclassed_count

In [14]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)


es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

model.fit(data_train, epochs=5, validation_data=data_valid, batch_size=128, callbacks=[es], class_weight=reclassed_dict)
# model.save("VGG10-200.h5")

Epoch 1/5


KeyboardInterrupt: ignored

In [ ]:
model.save("VGG20-244-3DNN.h5")

In [ ]:
pred_array = model.predict(data_test)
pred_class = [int(list(data_train.class_indices.keys())[i]) for i in pred_array.argmax(axis=-1)]
print('Test Data accuracy: ', accuracy_score(test_labels, pred_class)*100)

In [ ]:
from sklearn import preprocessing
normalized_arr = preprocessing.normalize(pred_array)

# print(normalized_arr[0])

scaled_df = pd.DataFrame(pred_array)
# scaled_df.round(5)
scaled_df[0:18]

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
1,2.483704e-17,1.000000e+00,1.016866e-19,8.650681e-16,6.294549e-14,1.343606e-14,2.944306e-21,6.562075e-20,9.488334e-26,4.323303e-30,...,1.283403e-24,4.544544e-15,1.409487e-14,3.650019e-20,2.747322e-25,7.743994e-10,1.803074e-15,1.086450e-16,4.851673e-21,1.368877e-17
2,9.795411e-10,3.737615e-13,1.205220e-08,4.779156e-15,8.515151e-15,2.075278e-17,1.457805e-22,6.850915e-16,6.201889e-14,9.793098e-24,...,1.048860e-08,1.604150e-07,9.693179e-12,3.696021e-21,6.185530e-17,9.524000e-06,8.832891e-17,4.036451e-08,9.999900e-01,1.635704e-17
3,1.482289e-14,2.562395e-08,5.159306e-08,5.179025e-08,9.813411e-01,1.856656e-02,1.119208e-11,8.429241e-10,7.610420e-18,1.096452e-07,...,9.008550e-10,2.838593e-07,2.571021e-08,2.761367e-08,2.822451e-10,8.856486e-08,3.839449e-12,3.213816e-08,2.000281e-09,5.539090e-07
4,5.081746e-12,9.690204e-11,3.713485e-15,1.560842e-01,6.212715e-08,6.490804e-13,1.702903e-15,7.055254e-19,2.078421e-19,4.116316e-14,...,3.578982e-12,2.387305e-13,9.010096e-16,1.081387e-14,7.836794e-18,3.361730e-10,1.382295e-14,8.395087e-11,4.255574e-12,1.130752e-13
5,8.242413e-08,9.079920e-10,2.651767e-11,3.029832e-12,5.028503e-11,2.029331e-16,3.782675e-19,3.121286e-15,1.287639e-11,1.247333e-17,...,3.069523e-04,8.777192e-10,9.170306e-13,1.818691e-18,4.309638e-15,3.401850e-05,5.608323e-14,3.182259e-04,9.993407e-01,1.207520e-15
6,1.168498e-24,9.968022e-21,1.324777e-26,5.837486e-17,2.260863e-29,9.848741e-27,1.923522e-26,4.680529e-30,0.000000e+00,5.336182e-31,...,6.596175e-30,2.148803e-24,7.468435e-29,1.658405e-36,1.640830e-35,6.946098e-28,2.367406e-35,2.628665e-23,1.392820e-27,3.204907e-25
7,6.182154e-16,3.820690e-09,1.091570e-07,6.370724e-09,9.998023e-01,1.603900e-04,2.992281e-14,1.666908e-10,9.473456e-15,7.078474e-08,...,1.408471e-08,2.914311e-09,2.711769e-06,5.056348e-07,1.285065e-08,3.923978e-08,2.267105e-10,2.662848e-10,1.671554e-09,5.356490e-08
8,5.301835e-16,1.791485e-11,1.604372e-09,4.070692e-06,1.378409e-03,1.882002e-08,1.566940e-14,9.383761e-14,2.249315e-19,5.262691e-13,...,2.011580e-10,1.854029e-09,1.897441e-10,1.383454e-12,4.952445e-14,2.734707e-11,1.875019e-12,1.732006e-12,4.344626e-11,2.732857e-10
9,7.936044e-35,3.945220e-21,1.425537e-21,4.889904e-29,6.877599e-25,9.182113e-22,2.309024e-30,1.257659e-19,0.000000e+00,1.562094e-33,...,1.124075e-23,2.346458e-13,1.818168e-17,4.463197e-34,6.051179e-31,1.903604e-28,1.176039e-33,8.201789e-25,1.370081e-26,1.520572e-27
10,3.197489e-16,1.738112e-10,3.410333e-11,1.148686e-09,6.723639e-02,1.462126e-06,6.375218e-11,2.502973e-13,2.868833e-18,2.168808e-06,...,3.346402e-10,1.133015e-12,1.013480e-09,1.816426e-09,2.198684e-10,1.212949e-10,9.408808e-13,2.916399e-12,2.931420e-14,3.306811e-09


In [ ]:
results_pd = pd.DataFrame({'True Label': test_labels})
prob_truth = []
prob_pred  = []

# for i,j in zip(data_keys, label_order):
#     reclassed_count[j] = label_count[int(i)]
#     prob_truth.append(pred_array[i][test_labels[0]]) 
#     prob_pred.append(pred_array[i][pred_class[0]]) 

int_keys = [int(int_string) for int_string in data_keys]


for i in range(pred_array.shape[0]): 
  # prob_pred.append(pred_array[i][pred_class[0]]) 
  prob_truth.append(pred_array[int_keys.index(test_labels[i])])
  prob_pred.append(pred_array[int_keys.index(pred_class[i])]) 

results_pd['True Prob']  = prob_truth
results_pd['Prediction'] = pred_class
results_pd['Pred Prob']  = prob_pred

results_pd[0:20]

,True Label,True Prob,Prediction,Pred Prob
0,16,6.550492e-12,8,1.682443e-01
1,1,3.598762e-14,1,3.994656e-19
2,38,6.602150e-14,8,8.554371e-13
3,33,2.140436e-07,13,2.043278e-16
4,11,2.821047e-05,30,1.072926e-25
5,38,6.131848e-19,8,7.510450e-15
6,18,1.644114e-15,18,1.194285e-32
7,12,1.548071e-06,12,6.968719e-18
8,25,1.596507e-03,25,7.003813e-15
9,35,1.494036e-30,35,3.207589e-37


In [ ]:
data_train.class_indices.keys()

dict_keys(['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '4', '40', '41', '42', '5', '6', '7', '8', '9'])